In [1]:
# Import Python packages 
import pandas as pd
import numpy as np

import cassandra
from cassandra.cluster import Cluster

import re
import os
import glob

import json
import csv

# Cassandra
# https://medium.com/@manishyadavv/how-to-install-cassandra-on-mac-os-d9338fcfcba4
# cassandra -f

In [2]:
# current working directory
print(os.getcwd())

# add subfolder event_data to current 
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/pierre-olivierboisvert/Desktop/GIT/GroupMiniCassandraETL


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file in utf-8 encoding
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile)
        # go to next file
        next(csvreader)
        
 # extracting each row one by one and append to full_data_rows_list        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Document inspection - Total rows
print(len(full_data_rows_list))

# create smaller data csv file called event_datafile_full csv
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_data_etl.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows of the new data file
with open('event_data_etl.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# Establish connection and begin executing queries, need a session

try:
    cluster = Cluster(['127.0.0.1'])
    
    session = cluster.connect()

except Exception as e:
    print(e)

In [6]:
# Creating the keyspace sesion

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandradb 
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
    
except Exception as e:
    print(e)

In [7]:
# connecting to DB

try:
    session.set_keyspace('cassandradb')
except Exception as e:
    print(e)

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + '(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))'

try:
    session.execute(query)
    print('Table session_song created')

except Exception as e:
    print(e)

Table session_song created


In [9]:
# Checking if the table is empty
try:
    rows = session.execute('SELECT COUNT(*) FROM session_songs')

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(count=6820)


In [10]:
# Adding some data into the new table
csv_file = 'event_data.csv'

query = "INSERT INTO session_songs (sessionId, itemInsession, artist, song, length) VALUES (%s, %s, %s, %s, %s)"

# reading csv file 
with open(csv_file, 'r', encoding = 'utf8', newline='') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
    next(csvreader)

    # Reading each line and adding it to the table
    for line in csvreader:
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        try:
            session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))

        except Exception as e:
            print(f'The line was not inserted\nError: {e}')

#checking the inserted data
try:
    rows = session.execute('SELECT COUNT(*) FROM session_songs')

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(count=6820)


In [11]:
# Answering the question "Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4"

try:
    rows = session.execute("SELECT artist, song, length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4")
    
    for row in rows:
        print(f'Artist: {row.artist}, song name: {row.song}, length: {row.length}')
    
except Exception as e:
    print(e)

Artist: Faithless, song name: Music Matters (Mark Knight Dub), length: 495.30731201171875


In [12]:
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + '( userId int, sessionId int, itemInSession int, artist text, \
                   song text, firstName text, lastName text, \
                   PRIMARY KEY((userId, sessionId), itemInSession))'

try:
    session.execute(query)
    print('Table user_session created')

except Exception as e:
    print(e)

Table user_session created


In [13]:
# Checking if the table is empty
try:
    rows = session.execute('SELECT COUNT(*) FROM user_session')

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(count=0)


In [14]:
# Adding some data into the new table
csv_file = 'event_data_etl.csv'
query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s)"

# reading csv file 
with open(csv_file, 'r', encoding = 'utf8', newline='') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
    next(csvreader)

    # Reading each line and adding it to the table
    for line in csvreader:
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        try:
            session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song, firstName, lastName))

        except Exception as e:
            print(f'The line was not inserted\nError: {e}')

In [15]:
#checking the inserted data

try:
    rows = session.execute('SELECT COUNT(*) FROM user_session')

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(count=6820)


In [16]:
# Answering the question "Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182"

try:
    rows = session.execute("SELECT artist, song, firstName, lastName \
                           FROM user_session \
                           WHERE userId = 10 AND sessionId = 182 \
                           ORDER BY itemInSession" )
    
    for row in rows:
        print(f'Artist: {row.artist}, Song name: {row.song}, User: {row.firstname} {row.lastname}')
        
except Exception as e:
    print(e)

Artist: Down To The Bone, Song name: Keep On Keepin' On, User: Sylvie Cruz
Artist: Three Drives, Song name: Greece 2000, User: Sylvie Cruz
Artist: Sebastien Tellier, Song name: Kilometer, User: Sylvie Cruz
Artist: Lonnie Gordon, Song name: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), User: Sylvie Cruz


In [17]:
query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + '(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))'

try:
    session.execute(query)
    print('Table user_songs created')

except Exception as e:
    print(e)

Table user_songs created


In [18]:
# Checking if the table is empty
try:
    rows = session.execute('SELECT COUNT(*) FROM user_songs')

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(count=0)


In [20]:
# Adding some data into the new table
csv_file = 'event_data_etl.csv'
query = "INSERT INTO user_songs (song, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"

# reading csv file 
with open(csv_file, 'r', encoding = 'utf8', newline='') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
    next(csvreader)

    # Reading each line and adding it to the table
    for line in csvreader:
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        try:
            session.execute(query, (song, int(userId), firstName, lastName))

        except Exception as e:
            print(f'The line was not inserted\nError: {e}')

In [21]:
#checking the inserted data

try:
    rows = session.execute('SELECT COUNT(*) FROM user_songs')

except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(count=6618)


In [22]:
# Get History - who listened to the song 'All Hands Against His Own'

try:
    rows = session.execute("SELECT firstName, lastName\
                           FROM user_songs \
                           WHERE song = 'All Hands Against His Own'" )
    
    for row in rows:
        print(f'User: {row.firstname} {row.lastname}')
        
except Exception as e:
    print(e)

User: Jacqueline Lynch
User: Tegan Levine
User: Sara Johnson


### Delete DB

In [23]:
# All tables in a list
queries_drop = ["DROP TABLE session_songs", 
                "DROP TABLE user_session ", 
                "DROP TABLE user_songs"]

try:
    for query in queries_drop:
        rows = session.execute(query)
        
except Exception as e:
    print(e)